In [1]:
from financiero_3 import *
# import plotly.io as pio
# pio.renderers.default = "notebook"

In [2]:
#Fondo de reserva general
fondo_reserva = fondo_reserva_general()

Última actualización: 30-07-2025
Cantidad del fondo requerida: 4297€


In [3]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.25.0
ipykernel        : 6.28.0
ipywidgets       : 7.8.1
jupyter_client   : 8.6.0
jupyter_core     : 5.7.2
jupyter_server   : 2.14.1
jupyterlab       : 4.0.11
nbclient         : 0.10.2
nbconvert        : 7.10.0
nbformat         : 5.9.2
notebook         : 7.0.8
qtconsole        : not installed
traitlets        : 5.14.3


In [ ]:
# Lectura de datos
excel = os.getcwd() + '/Data/Registros/' + "Inicio.xlsx"

gastos = pd.read_excel(excel, sheet_name=0).iloc[:, [0, 1, 2, 3] + [-2, -1]]
ingresos = pd.read_excel(excel, sheet_name=1).iloc[:, [0, 1, 2, 3] + [-2, -1]]
transferencias = pd.read_excel(excel, sheet_name=2).drop(columns=['Cantidad en divisa entrante', 'Divisa saliente', 'Divisa entrante']).iloc[:-1]

gastos.columns = ["fecha", "categoria", "cuenta", "cantidad", "etiquetas", "comentario"]
ingresos.columns = ["fecha", "categoria", "cuenta", "cantidad", "etiquetas", "comentario"]
transferencias.columns = ["fecha", "saliente", "entrante", "cantidad", "comentario"]

gastos['categoria'] = gastos['categoria'].apply(eliminar_tildes)
ingresos['categoria'] = ingresos['categoria'].apply(eliminar_tildes)

ingresos['tipo_logico'] = ingresos.apply(clasificar_ingreso, axis=1)
gastos['tipo_logico'] = gastos.apply(clasificar_ingreso, axis=1)
# Convertir fechas a formato datetime
for df in [gastos, ingresos, transferencias]:
    df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

# Extraer nombres de cuentas
cuentas = set(gastos['cuenta']).union(ingresos['cuenta']).union(transferencias['saliente']).union(transferencias['entrante'])

# Crear objetos Account
cuentas_obj = {nombre: Account(nombre) for nombre in cuentas}

# Crear presupuesto y añadir cuentas
presupuesto = Budget()
for cuenta in cuentas_obj.values():
    presupuesto.add_account(cuenta)




cargar_saldos_iniciales(presupuesto, cuentas_obj, saldos_iniciales)


# Cargar ingresos (+) y gastos (-)
cargar_transacciones(ingresos, cuentas_obj, presupuesto, signo=1)
cargar_transacciones(gastos, cuentas_obj, presupuesto, signo=-1)

for _, fila in transferencias.iterrows():
    fecha = fila['fecha']
    cantidad = fila['cantidad']
    comentario = str(fila['comentario'])

    cuenta_origen = cuentas_obj[fila['saliente']]
    cuenta_destino = cuentas_obj[fila['entrante']]

    trans_salida = Transaction(
        amount=-cantidad,
        description=f"[TRANSFERENCIA SALIENTE] {comentario}",
        category="Transferencia",
        account=cuenta_origen,
        timestamp=fecha
    )

    trans_entrada = Transaction(
        amount=cantidad,
        description=f"[TRANSFERENCIA ENTRANTE] {comentario}",
        category="Transferencia",
        account=cuenta_destino,
        timestamp=fecha
    )

    presupuesto.add_transaction(trans_salida)
    presupuesto.add_transaction(trans_entrada)


print("Saldo total:", presupuesto.get_balance())

for nombre, cuenta in cuentas_obj.items():
    print(f"Cuenta: {nombre}, Saldo: {round(cuenta.balance,2)}")

ingresos = procesar_ingresos(ingresos)

print("💼 Resumen de ingresos:")
print(resumen_ingresos(ingresos))

print("\n📉 Gastos personales netos:")
print(calcular_gastos_netos(gastos, ingresos))

print("\n📅 Resumen mensual:")
print(resumen_mensual(gastos, ingresos))

# Procesar y calcular
ingresos = procesar_ingresos(ingresos)
gastos_netos = calcular_gastos_netos(gastos, ingresos)
resumen = resumen_mensual(gastos, ingresos)

# Visualizaciones
plot_balance_mensual(resumen)
plot_explicacion_gastos(gastos,ingresos)

# Porcentaje de ahorro
plot_porcentaje_ahorro(gastos, ingresos)

historial = crear_historial_cuentas_virtuales(ingresos, gastos, presupuesto, porcentaje_gasto=0.3)
historial

Saldo total: 21549.459999999995
Cuenta: Ahorro, Saldo: 10440.06
Cuenta: Metálico, Saldo: 33.01
Cuenta: Amex, Saldo: -502.59
Cuenta: Revolut, Saldo: -0.0
Cuenta: Principal, Saldo: 442.0
Cuenta: Invertido, Saldo: 1300.0
Cuenta: Cobee, Saldo: -398.7
Cuenta: My Investor, Saldo: 10235.68
💼 Resumen de ingresos:
24099.66

📉 Gastos personales netos:
categoria
Alimentacion        183.05
Entretenimiento    1723.66
Hosteleria         3618.44
Otros              1701.83
Transporte         2823.17
Name: cantidad, dtype: float64

📅 Resumen mensual:
grupo    ingresos   gastos  balance
mes                                
2024-10   1717.37   730.48   986.89
2024-11   1699.06   626.10  1072.96
2024-12   2921.28   656.92  2264.36
2025-01   1452.31   457.82   994.49
2025-02   1587.22  1054.48   532.74
2025-03   2292.46   670.40  1622.06
2025-04   2232.59   383.10  1849.49
2025-05   1572.41   551.72  1020.69
2025-06   1465.48  1052.28   413.20
2025-07   3387.51   594.77  2792.74
2025-08   1481.71   226.72  

In [ ]:
365.0+114.74+400.11+1150.0+18268.09 -21644.3

In [ ]:
resumen_global(presupuesto, "2025-09-30")

In [ ]:
presupuesto

In [ ]:
presupuesto.balances_a_fecha(pd.to_datetime("2025-08-30"))["Invertido"]

In [ ]:
predecir_total(historial, 'Deuda Presupuestaria acumulada')

In [ ]:
grafico_presupuesto(historial)

In [ ]:
graficar_gasto_vs_media(gastos, ingresos)

In [ ]:
# ultimo_mes = df_resumen['Mes'].max().to_period('M')
pd.Timestamp.today().to_period('M')